In [3]:
# !pip install connected-components-3d
# !pip install vtk
# !pip install SimpleITK
# !pip uninstall vtk
# !pip install vtk
# !pip install vtk
# !pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk
import time
import cv2

# Define paths to segmentation data
nrrd_paths = [
    "label0004.nii.gz"
    # "mlp_model_data/468_day7_mask_combined_c_converted.nii.gz",
    # "mlp_model_data/468_day7_mask_combined_d_converted.nii.gz",
    # "mlp_model_data/468_day7_mask_combined_a_converted.nii.gz",

    # Add more paths as needed
]

# Screen dimensions
screen_width = 1280
screen_height = 720

# Determine the number of rows and columns
num_cells = len(nrrd_paths)
rows = int(np.ceil(np.sqrt(num_cells)))
cols = int(np.ceil(num_cells / rows))

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Connected Components 3D Visualization")
render_window.SetSize(screen_width, screen_height)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
max_label = 0

# Increase spacing between cells
cell_spacing_x = screen_width / (cols + 1) * 1.5  # Increase the multiplier for more space
cell_spacing_y = screen_height / (rows + 1) * 1.5  # Increase the multiplier for more space

# Scaling factor
scaling_factor = 2.0  # Adjust this factor to scale the cells if needed

# Process each segmentation file
for idx, nrrd_path in enumerate(nrrd_paths):
    # Load your segmentation data
    image = sitk.ReadImage(nrrd_path)
    labels_in = sitk.GetArrayFromImage(image)

    # Ensure that labels are consecutive integers starting from 1
    labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 26-connected
    max_label = max(max_label, np.max(labels_out))

    # Create a VTK image data object
    vtk_image = vtk.vtkImageData()
    vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
    vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

    # Copy NumPy array data to VTK image
    vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
    vtk_image.GetPointData().SetScalars(vtk_array)

    # Create a VTK Discrete Marching Cubes algorithm
    discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
    discrete_marching_cubes.SetInputData(vtk_image)
    discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

    # Apply smoothing
    smooth_filter = vtk.vtkSmoothPolyDataFilter()
    smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
    smooth_filter.SetNumberOfIterations(500)  # Adjust the number of iterations as needed

    # Assign different colors to each connected component label
    lut.SetNumberOfTableValues(max_label + 1)
    lut.Build()
    for i in range(1, np.max(labels_out) + 1):
        color = np.random.rand(3)  # Random RGB color
        lut.SetTableValue(i, color[0], color[1], color[2])

    # Create a VTK PolyDataMapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
    mapper.SetLookupTable(lut)
    mapper.SetScalarRange(0, np.max(labels_out))

    # Create a VTK actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    
    # Calculate position based on row and column
    row = idx // cols
    col = idx % cols
    x = (col + 1) * cell_spacing_x - screen_width / 2
    y = (row + 1) * cell_spacing_y - screen_height / 2
    z = 0  # Adjust Z offset if needed

    # Apply transformation to position each actor
    transform = vtk.vtkTransform()
    transform.Translate(x, y, z)
    transform.Scale(scaling_factor, scaling_factor, scaling_factor)  # Scale the cells
    actor.SetUserTransform(transform)

    # Add the actor to the renderer
    renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(1.0, 1.0, 1.0)  # Light gray background
renderer.ResetCamera()

# Add axes to the scene
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.0, 0.8, 0.2, 1.0)
# Set up video writer using OpenCV
fps = 20  # Capture frames at a higher FPS for smoother video
frame_width = 1280
frame_height = 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('10a_day7_c.mp4', fourcc, fps, (frame_width, frame_height))

# Start the rendering loop with faster camera rotation
start_time = time.time()
rotation_speed = 1  # Increase this value for faster camera rotation
frame_interval = 1 / fps  # Time between frames
while time.time() - start_time < 60:
    render_window.Render()
    
    # Capture the frame from the render window
    w2if = vtk.vtkWindowToImageFilter()
    w2if.SetInput(render_window)
    w2if.Update()
    
    vtk_image = w2if.GetOutput()
    vtk_array = vtk_image.GetPointData().GetScalars()
    frame = numpy_support.vtk_to_numpy(vtk_array).reshape((frame_height, frame_width, -1))
    frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
    
    # Write the frame to the video file
    video_writer.write(frame)
    
    # Rotate the camera faster
    renderer.GetActiveCamera().Azimuth(rotation_speed)  # Rotate faster
    render_window.Render()
    time.sleep(frame_interval)  # Maintain the FPS

# Release the video writer
video_writer.release()

# Start the interactor to show the window
render_window_interactor.Start()


In [ ]:
##In crease Resulation

In [10]:
#Save as 3d file for vuejs

In [2]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk
import time

# Load your segmentation data
nrrd_path = "mlp_model_data/img0001.nii_label.nii"
image = sitk.ReadImage(nrrd_path)
labels_in = sitk.GetArrayFromImage(image)

# Ensure that labels are consecutive integers starting from 1
labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

# Create a VTK image data object
vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

# Copy NumPy array data to VTK image
vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
vtk_image.GetPointData().SetScalars(vtk_array)

# Create a VTK Discrete Marching Cubes algorithm
discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
discrete_marching_cubes.SetInputData(vtk_image)
discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

# Apply smoothing
smooth_filter = vtk.vtkSmoothPolyDataFilter()
smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
smooth_filter.SetNumberOfIterations(500)  # Adjust the number of iterations as needed

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Connected Components 3D Visualization")
render_window.SetSize(800, 800)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(np.max(labels_out) + 1)
lut.Build()

# Assign different colors to each connected component label
for i in range(1, np.max(labels_out) + 1):
    color = np.random.rand(3)  # Random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2])

# Create a VTK PolyDataMapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
mapper.SetLookupTable(lut)
mapper.SetScalarRange(0, np.max(labels_out))

# Create a VTK actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(1.0, 1.0, 1.0)  # Light gray background
renderer.ResetCamera()

# Add axes to the scene
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.8, 0.0, 1.0,1.0)  # Adjust the viewport position and size

# Start the rendering loop
render_window.Render()

# Rotate the scene for 1 minute
start_time = time.time()
while time.time() - start_time < 60:
    renderer.GetActiveCamera().Azimuth(1)  # Rotate the camera azimuth
    renderer.GetActiveCamera().Elevation(0.5)  # Rotate the camera elevation
    renderer.GetActiveCamera().Roll(0.2)  # Rotate the camera roll
    render_window.Render()
    time.sleep(0.01)  # Adjust the sleep time for smooth rotation

render_window_interactor.Start()


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /Users/runner/work/SimpleITK/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "mlp_model_data/img0001.nii_label.nii" does not exist.

In [6]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk

# Load your segmentation data
nrrd_path = "mlp_model_data/img0001.nii_label.nii"
image = sitk.ReadImage(nrrd_path)
labels_in = sitk.GetArrayFromImage(image)

# Ensure that labels are consecutive integers starting from 1
labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

# Create a VTK image data object
vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

# Copy NumPy array data to VTK image
vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
vtk_image.GetPointData().SetScalars(vtk_array)

# Create a VTK Discrete Marching Cubes algorithm
discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
discrete_marching_cubes.SetInputData(vtk_image)
discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

# Apply smoothing
smooth_filter = vtk.vtkSmoothPolyDataFilter()
smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
smooth_filter.SetNumberOfIterations(1000)  # Adjust the number of iterations as needed: sMOOTH

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(np.max(labels_out) + 1)
lut.Build()

# Assign different colors to each connected component label
for i in range(1, np.max(labels_out) + 1):
    color = np.random.rand(3)  # Random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2])

# Create a VTK PolyDataMapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
mapper.SetLookupTable(lut)
mapper.SetScalarRange(0, np.max(labels_out))

# Create a VTK actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Save colored polydata to a VTK.vtp file
vtp_writer = vtk.vtkXMLPolyDataWriter()
vtp_writer.SetFileName("connected_components_colored.vtp")
vtp_writer.SetInputConnection(smooth_filter.GetOutputPort())
vtp_writer.Write()

# ... (rest of the script remains unchanged)


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk\Code\IO\src\sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "mlp_model_data/img0035_label (1).nii" does not exist.

In [1]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk
import nibabel as nib

# Load your segmentation data
nii_path = "10a_day5_mask_combined_c_converted.nii.gz"
img = nib.load(nii_path)
labels_in = img.get_fdata()

# Ensure that labels are consecutive integers starting from 1
labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

# Create a VTK image data object
vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

# Copy NumPy array data to VTK image
vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
vtk_image.GetPointData().SetScalars(vtk_array)

# Create a VTK Discrete Marching Cubes algorithm
discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
discrete_marching_cubes.SetInputData(vtk_image)
discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

# Apply smoothing
smooth_filter = vtk.vtkSmoothPolyDataFilter()
smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
smooth_filter.SetNumberOfIterations(700)  # Adjust the number of iterations as needed: sMOOTH

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Connected Components 3D Visualization")
render_window.SetSize(800, 800)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(np.max(labels_out) + 1)
lut.Build()

# Assign different colors to each connected component label
for i in range(1, np.max(labels_out) + 1):
    color = np.random.rand(3)  # Random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2])

# Create a VTK PolyDataMapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
mapper.SetLookupTable(lut)
mapper.SetScalarRange(0, np.max(labels_out))

# Create a VTK actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(255, 255, 255)  # Light gray background
renderer.ResetCamera()

# Add axes to the scene
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.8, 0.0, 1.0, 0.2)  # Adjust the viewport position and size

# Start the rendering loop
render_window.Render()
render_window_interactor.Start()


In [ ]:
#Multple converted
# !pip install vtk


In [18]:
import os
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk

# Directory containing the NII.gz files
directory = "mlp_model_data/synthetic_3d_image_00_flattened_ellipsoids.nii.gz"

# Output directory for VTP files
output_directory = "Untitled Folder 3"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".nii.gz"):
        filepath = os.path.join(directory, filename)

        # Load segmentation data
        image = sitk.ReadImage(filepath)
        labels_in = sitk.GetArrayFromImage(image)

        # Ensure that labels are consecutive integers starting from 1
        labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

        # Create a VTK image data object
        vtk_image = vtk.vtkImageData()
        vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
        vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

        # Copy NumPy array data to VTK image
        vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
        vtk_image.GetPointData().SetScalars(vtk_array)

        # Create a VTK Discrete Marching Cubes algorithm
        discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
        discrete_marching_cubes.SetInputData(vtk_image)
        discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

        # Apply smoothing
        smooth_filter = vtk.vtkSmoothPolyDataFilter()
        smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
        smooth_filter.SetNumberOfIterations(500)  # Adjust the number of iterations as needed: sMOOTH

        # Create a VTK lookup table
        lut = vtk.vtkLookupTable()
        lut.SetNumberOfTableValues(np.max(labels_out) + 1)
        lut.Build()

        # Assign different colors to each connected component label
        for i in range(1, np.max(labels_out) + 1):
            color = np.random.rand(3)  # Random RGB color
            lut.SetTableValue(i, color[0], color[1], color[2])

        # Create a VTK PolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
        mapper.SetLookupTable(lut)
        mapper.SetScalarRange(0, np.max(labels_out))

        # Save colored polydata to a VTK.vtp file with a unique name
        vtp_filename = f"{os.path.splitext(filename)[0]}_connected_components_colored.vtp"
        vtp_filepath = os.path.join(output_directory, vtp_filename)
        vtp_writer = vtk.vtkXMLPolyDataWriter()
        vtp_writer.SetFileName(vtp_filepath)
        vtp_writer.SetInputConnection(smooth_filter.GetOutputPort())
        vtp_writer.Write()


NotADirectoryError: [WinError 267] The directory name is invalid: 'mlp_model_data/synthetic_3d_image_00_flattened_ellipsoids.nii.gz'

In [1]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk
import nibabel as nib

# Load your segmentation data
nii_path = "label0001.nii.gz"
img = nib.load(nii_path)
labels_in = img.get_fdata()

# Ensure that labels are consecutive integers starting from 1
labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

# Create a VTK image data object
vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

# Copy NumPy array data to VTK image
vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
vtk_image.GetPointData().SetScalars(vtk_array)

# Create a VTK Discrete Marching Cubes algorithm
discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
discrete_marching_cubes.SetInputData(vtk_image)
discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

# Apply smoothing
smooth_filter = vtk.vtkSmoothPolyDataFilter()
smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
smooth_filter.SetNumberOfIterations(400)  # Adjust the number of iterations as needed: sMOOTH

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Connected Components 3D Visualization")
render_window.SetSize(800, 800)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(np.max(labels_out) + 1)
lut.Build()

# Assign different colors to each connected component label
for i in range(1, np.max(labels_out) + 1):
    color = np.random.rand(3)  # Random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2])

# Create a VTK PolyDataMapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
mapper.SetLookupTable(lut)
mapper.SetScalarRange(0, np.max(labels_out))

# Create a VTK actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(255, 255, 255)  # Light gray background
renderer.ResetCamera()

# Add axes to the scene
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.8, 0.0, 1.0, 0.2)  # Adjust the viewport position and size

# Start the rendering loop
render_window.Render()
render_window_interactor.Start()


In [5]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk

# Define paths to segmentation data
nrrd_paths = [
    "10a_day7_mask_combined_c_converted.nii.gz"
    # Add more paths as needed
]

# Process each segmentation file
for idx, nrrd_path in enumerate(nrrd_paths):
    # Load your segmentation data
    image = sitk.ReadImage(nrrd_path)
    labels_in = sitk.GetArrayFromImage(image)

    # Ensure that labels are consecutive integers starting from 1
    labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 26-connected

    # Create a VTK image data object
    vtk_image = vtk.vtkImageData()
    vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
    vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

    # Copy NumPy array data to VTK image
    vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
    vtk_image.GetPointData().SetScalars(vtk_array)

    # Create a VTK Discrete Marching Cubes algorithm
    discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
    discrete_marching_cubes.SetInputData(vtk_image)
    discrete_marching_cubes.GenerateValues(int(np.max(labels_out)), 1, int(np.max(labels_out)))
    discrete_marching_cubes.Update()

    # Get the polydata from the marching cubes output
    polydata = discrete_marching_cubes.GetOutput()

    # Ensure the polydata has scalars
    scalars = polydata.GetCellData().GetScalars()
    if scalars is None:
        print("Error: No scalars found in polydata.")
        continue

    # Convert the cell scalars to point scalars
    cell_scalars = numpy_support.vtk_to_numpy(scalars).astype(int)
    num_labels = np.max(cell_scalars)

    # Generate random colors for each label
    colors = np.random.randint(0, 256, size=(num_labels + 1, 3), dtype=np.uint8)

    # Create a VTK array to store colors
    color_array = vtk.vtkUnsignedCharArray()
    color_array.SetNumberOfComponents(3)
    color_array.SetName("Colors")

    # Assign colors to each cell based on the label
    for label in cell_scalars:
        color_array.InsertNextTuple3(*colors[label])

    # Add the color array to the polydata
    polydata.GetCellData().SetScalars(color_array)

    # Create a PLY writer to save the output
    ply_writer = vtk.vtkPLYWriter()
    ply_writer.SetFileName(f"output_colored_{idx}.ply")
    ply_writer.SetInputData(polydata)
    ply_writer.SetFileTypeToASCII()  # Use SetFileTypeToBinary() for binary output
    ply_writer.Write()

    print(f"Colored PLY file saved as output_colored_{idx}.ply")

print("Conversion complete.")


Colored PLY file saved as output_colored_0.ply
Conversion complete.


In [1]:
import vtk
import numpy as np
import time

# Path to your .vtp file
vtp_file_path = "img0002.nii_output_5000.vtp"

# Screen dimensions
screen_width = 1280
screen_height = 720

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("VTP File with Assigned Colors")
render_window.SetSize(screen_width, screen_height)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Load the .vtp file
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(vtp_file_path)
reader.Update()

# Get the output data
output_data = reader.GetOutput()

# Debug: List available data arrays
print("Available PointData arrays:")
for i in range(output_data.GetPointData().GetNumberOfArrays()):
    print(f"Array {i}: {output_data.GetPointData().GetArrayName(i)}")

# Try to get scalar data
scalars = output_data.GetPointData().GetScalars()
if scalars is None:
    print("No scalar data found in PointData. Checking CellData...")
    for i in range(output_data.GetCellData().GetNumberOfArrays()):
        print(f"Cell Array {i}: {output_data.GetCellData().GetArrayName(i)}")
    scalars = output_data.GetCellData().GetScalars()

if scalars is None:
    print("No scalar data found in the .vtp file. Cannot proceed with coloring.")
    exit()

# Get the scalar range
scalar_range = scalars.GetRange()
print(f"Scalar Range: {scalar_range}")

# Create a lookup table to assign different colors to each component
lut = vtk.vtkLookupTable()
num_components = int(scalar_range[1])  # Maximum component ID
lut.SetNumberOfTableValues(num_components + 1)  # Add one for 0 index if needed
lut.Build()

# Assign random colors to each component
for i in range(num_components + 1):
    color = np.random.rand(3)  # Generate random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2], 1.0)  # RGB with full opacity

# Set up the mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())
mapper.SetLookupTable(lut)
mapper.SetScalarRange(scalar_range)  # Use scalar range to map colors
mapper.ScalarVisibilityOn()

# Create an actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(1.0, 1.0, 1.0)  # Light gray background
renderer.ResetCamera()

# Add axes orientation marker
# Add axes orientation marker
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)

# Correct viewport position (e.g., bottom-right corner)
axes_widget.SetViewport(0.8, 0.9, 0.7, 0.2)  # Bottom-right corner with proper dimensions

axes_widget.SetEnabled(1)
axes_widget.InteractiveOn()


axes_widget.SetEnabled(1)
axes_widget.InteractiveOn()

# Rotate the camera
rotation_speed = 0.2  # Degrees per frame

# Start the rendering loop
start_time = time.time()
while time.time() - start_time < 10:  # Rotate for 10 seconds
    render_window.Render()
    renderer.GetActiveCamera().Azimuth(rotation_speed)  # Rotate the camera

# Start the interactor to show the window
render_window_interactor.Start()


Available PointData arrays:
No scalar data found in PointData. Checking CellData...
Cell Array 0: Scalars
Scalar Range: (1.0, 20.0)


In [ ]:
import vtk
import numpy as np

# Path to your .vtp file
vtp_file_path = "img0002.nii_output.vtp"

# Screen dimensions
screen_width = 1280
screen_height = 720

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Interactive VTP Viewer")
render_window.SetSize(screen_width, screen_height)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Load the .vtp file
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(vtp_file_path)
reader.Update()

# Get the output data
output_data = reader.GetOutput()

# Debug: List available data arrays
print("Available PointData arrays:")
for i in range(output_data.GetPointData().GetNumberOfArrays()):
    print(f"Array {i}: {output_data.GetPointData().GetArrayName(i)}")

# Try to get scalar data
scalars = output_data.GetPointData().GetScalars()
if scalars is None:
    print("No scalar data found in PointData. Checking CellData...")
    for i in range(output_data.GetCellData().GetNumberOfArrays()):
        print(f"Cell Array {i}: {output_data.GetCellData().GetArrayName(i)}")
    scalars = output_data.GetCellData().GetScalars()

if scalars is None:
    print("No scalar data found in the .vtp file. Cannot proceed with coloring.")
    exit()

# Get the scalar range
scalar_range = scalars.GetRange()
print(f"Scalar Range: {scalar_range}")

# Create a lookup table to assign different colors to each component
lut = vtk.vtkLookupTable()
num_components = int(scalar_range[1])  # Maximum component ID
lut.SetNumberOfTableValues(num_components + 1)  # Add one for 0 index if needed
lut.Build()

# Assign random colors to each component
for i in range(num_components + 1):
    color = np.random.rand(3)  # Generate random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2], 1.0)  # RGB with full opacity

# Set up the mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())
mapper.SetLookupTable(lut)
mapper.SetScalarRange(scalar_range)  # Use scalar range to map colors
mapper.ScalarVisibilityOn()

# Create an actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(1.0, 1.0, 1.0)  # Light gray background
renderer.ResetCamera()

# Add axes orientation marker
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.8, 0.0, 1.0, 0.2)
axes_widget.SetEnabled(1)
axes_widget.InteractiveOn()

# Callback function to rotate the camera
def rotate_camera(obj, event):
    renderer.GetActiveCamera().Azimuth(1)  # Rotate 1 degree per callback
    render_window.Render()

# Timer event for continuous camera rotation
render_window_interactor.AddObserver("TimerEvent", rotate_camera)
timer_id = render_window_interactor.CreateRepeatingTimer(10)  # Every 10ms

# Enable user interactions
render_window_interactor.Initialize()
render_window.Render()
render_window_interactor.Start()


Available PointData arrays:
No scalar data found in PointData. Checking CellData...
Cell Array 0: Scalars
Scalar Range: (1.0, 8.0)
